### Multi Agent Systems


- Creating Sub-Agents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [5]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama

# Load the model
model = ChatOllama(model="llama3.2:1b", 
                   temperature=0,
                   validate_model_on_init=True,
                   tools=[square_root],
                   seed=42)

# create subagents

subagent_1 = create_agent(
    model,
    tools=[square_root]
)

subagent_2 = create_agent(
    model,
    tools=[square]
)

- Creating Main Agent

In [6]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model,
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

- Testing

In [7]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [8]:
response

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='c4e6e44c-3f9a-4f75-aad9-c12759189674'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-23T11:55:47.4762108Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12262812200, 'load_duration': 10093887300, 'prompt_eval_count': 248, 'prompt_eval_duration': 1206096500, 'eval_count': 24, 'eval_duration': 938631900, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b4b10-8f94-7952-ab99-6af33300b092-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': '456'}, 'id': 'fd9bd38e-a93a-4b34-a6bc-1e36faef1066', 'type': 'tool_call'}], usage_metadata={'input_tokens': 248, 'output_tokens': 24, 'total_tokens': 272}),
  ToolMessage(content='The square root of 456.0 is approximately 21.354156504062622.', name='call_subagent_1', id='9ca29c4d-6a0d-4871-b3a5-b66306621e80', tool_c